# aws_a2i Usage Example

In [17]:
import os
import uuid
from pathlib import Path

from boto_session_manager import BotoSesManager

import aws_a2i

First, we define some constant values

In [5]:
# the AWS boto session manager
# see https://pypi.org/project/boto-session-manager/ for more information
bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_2")

dir_here = Path(os.getcwd()).absolute()

# where you store the task template (the .liquid file)
path_task_template = dir_here.parent.joinpath("task-ui.liquid")
# where you store the rendered task ui HTML
path_task_ui_html = dir_here.parent.joinpath("task-ui.html")

# the Ground Truth private team ARN
private_team_arn = (
    "arn:aws:sagemaker:us-east-2:669508176277:workteam/private-crowd/my-labeling-team"
)
# task UI template name
task_template_name = "aws-a2i-example"
# Human Review Workflow name
flow_definition_name = "aws-a2i-example"
# The IAM role for Human Review Workflow
flow_execution_role_arn = "arn:aws:iam::669508176277:role/a2i_poc-a2i_execution_role"
# Where you store HIL output data
output_bucket = "669508176277-us-east-2-a2i-poc"
output_key = "poc/aws_a2i/hil-output/"

Preview Task UI Template, in this example, we build a reimbursement request review system.

Given a reimbursement request, approve or deny it.

```
{
        "date_of_expense": "2020-01-01",
        "payment_method": "Credit Card",
        "purpose_of_expenditure": "Business Meal",
        "amount": "36.99",
}
```

In [9]:
print(path_task_template.read_text())

<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <full-instructions header="Instructions">
    <p>Check the following data points: Date of expense, Payment method, Purpose of expenditure, amount. </p>
  </full-instructions>

  <short-instructions>
    <p>Review this reimbursement request, approve or deny it.</p>
  </short-instructions>

  <h2>Reimbursement request</h2>
  <ul>
    <li>Date of expense = {{ task.input.date_of_expense }}</li>
    <li>Payment method = {{ task.input.payment_method }}</li>
    <li>Purpose of expenditure = {{ task.input.purpose_of_expenditure }}</li>
    <li>Amount = $ {{ task.input.amount }}</li>
  </ul>

  <h2>Approve or deny it</h2>

  <input type="radio" id="approve" name="decision" value="approve" required>
  <label for="approve">Approve</label>

  <input type="radio" id="deny" name="decision" value="deny" required>
  <label for="deny">Deny</label>

  <br>

</crowd-form>



Deploy Task UI Template

In [10]:
aws_a2i.deploy_hil_task_template(
    bsm=bsm,
    task_template_name=task_template_name,
    task_template_content=path_task_template.read_text(),
)

📋 Deploy Human in Loop task template
  preview at https://console.aws.amazon.com/a2i/home?region=us-east-2#/worker-task-templates/aws-a2i-example
  ✅ Successfully deployed task ui template 'aws-a2i-example'


Then we can deploy the Human Review Workflow definition

In [11]:
aws_a2i.deploy_flow_definition(
    bsm=bsm,
    flow_definition_name=flow_definition_name,
    flow_execution_role_arn=flow_execution_role_arn,
    labeling_team_arn=private_team_arn,
    output_bucket=output_bucket,
    output_key=output_key,
    task_template_name=task_template_name,
    task_description="Review reimbursement request",
    task_count=1,
)

📋 Deploy Human review workflow definition, it may takes 30 sec ~ 1 minute
  preview at https://console.aws.amazon.com/a2i/home?region=us-east-2#/human-review-workflows/aws-a2i-example
📋 Create Human review workflow definition ...
  on 1 th attempt, elapsed 1 seconds, remain 29 seconds ...
  on 2 th attempt, elapsed 2 seconds, remain 28 seconds ...
  ✅ Successfully deployed flow definition 'aws-a2i-example'


Now the human review workflow is ready to use, let's start some human loop task and review some reimbursement request!

In [16]:
workspace_signin_url = aws_a2i.get_workspace_signin_url(
    bsm=bsm,
    work_team_name=aws_a2i.parse_team_name_from_private_team_arn(arn=private_team_arn),
)
print(f"Login to the workspace: {workspace_signin_url}")

hil_arn = aws_a2i.start_human_loop(
    bsm=bsm,
    human_loop_name=str(uuid.uuid4()),
    flow_definition_arn=aws_a2i.get_flow_definition_arn(
        aws_account_id=bsm.aws_account_id,
        aws_region=bsm.aws_region,
        flow_definition_name=flow_definition_name,
    ),
    input_data={
        "date_of_expense": "2020-01-01",
        "payment_method": "Credit Card",
        "purpose_of_expenditure": "Business Meal",
        "amount": "36.99",
    },
)

Login to the workspace: https://3zqu42gydr.labeling.us-east-2.sagemaker.aws
▶️ Start a Human Loop Task 'd1908e9b-1ff1-4e33-8990-df99a21513e7'
  You can preview HIL status at https://us-east-2.console.aws.amazon.com/a2i/home?region=us-east-2#/human-review-workflows/aws-a2i-example/human-loops/d1908e9b-1ff1-4e33-8990-df99a21513e7


You can use the ``aws_a2i.render_task_template`` function to preview the task UI in your browser without starting a real human in the loop and without deploying anything. It is purely Python and purely local.

![](./task-ui.png)

In [14]:
input_data = {
    "date_of_expense": "2020-01-01",
    "payment_method": "Credit Card",
    "purpose_of_expenditure": "Business Meal",
    "amount": "36.99",
}
aws_a2i.render_task_template(
    task_template_content=path_task_template.read_text(),
    input_data=input_data,
    path_task_ui_html=path_task_ui_html,
    preview=True,
)

The usage example is done. Let's clean up AWS resources to avoid cost.

In [6]:
aws_a2i.delete_flow_definition(
    bsm=bsm,
    flow_definition_name=flow_definition_name,
)

{'ResponseMetadata': {'RequestId': '1bce4df2-56f2-4e97-9881-bb1d26083d6d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1bce4df2-56f2-4e97-9881-bb1d26083d6d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Sat, 18 Feb 2023 04:45:47 GMT'},
  'RetryAttempts': 0}}

In [7]:
aws_a2i.delete_human_task_ui(
    bsm=bsm,
    task_template_name=task_template_name,
)

{'ResponseMetadata': {'RequestId': '1cff9f1c-1656-46ba-b91f-ebab0fe17371',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1cff9f1c-1656-46ba-b91f-ebab0fe17371',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Sat, 18 Feb 2023 04:45:57 GMT'},
  'RetryAttempts': 0}}